In [ ]:
!pip install ultralytics

In [2]:
import os
import numpy as np
from ultralytics import YOLO
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
from google.colab import drive

drive.mount('/content/gdrive')

In [4]:
ROOT_DIR = '/content/gdrive/MyDrive/fish_data'

In [5]:
model_path = os.path.join(ROOT_DIR, 'results_on_100_epochs', 'runs', 'detect', 'train', 'weights', 'best.pt')

In [6]:
model = YOLO(model_path)

In [7]:
import cv2
from google.colab.patches import cv2_imshow

In [ ]:
image_path = '/content/gdrive/MyDrive/fish_data/test/images/image_with_aruko_marker_3.jpeg'
image = cv2.imread(image_path)
cv2_imshow(image)

In [ ]:
height, width, channels = image.shape
print(f'Height: {height} \n Width: {width}')

#Length Detection using Aruco marker

In [ ]:
image_predict = image.copy()
height, width, channels = image.shape

# detecting aruco marker
parameters = cv2.aruco.DetectorParameters()
aruco_dict = cv2.aruco.getPredefinedDictionary(cv2.aruco.DICT_5X5_50)
corners, _, _ = cv2.aruco.detectMarkers(image_predict, aruco_dict, parameters=parameters)

int_corners = np.int0(corners)
cv2.polylines(image_predict, int_corners, True, (225, 255, 0), 2)


#pixel-to-cm conversion
aruco_perimeter = cv2.arcLength(corners[0], True)
pixel_cm_ratio = aruco_perimeter / 20


# detecting fishes
results = model(image)[0]

threshold = 0.2
count = 0
for result in results.boxes.data.tolist():
  x1, y1, x2, y2, score, class_id = result

  object_length = abs((x1-x2)/pixel_cm_ratio)
  object_width = abs((y1-y2)/pixel_cm_ratio)

  if score > threshold:

      cv2.rectangle(image_predict, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 0), 2)
      #cv2.putText(image_predict, results.names[int(class_id)].upper(), (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
      cv2.putText(image_predict, "Length {} cm".format(round(object_length, 1)), (int(x1), int(y1 - 10)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)
      #cv2.putText(image_predict, "Width {} cm".format(round(object_width, 1)), (int(x1), int(y1 - 40)), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 1, cv2.LINE_AA)

      count = count + 1

cv2.putText(image_predict, "Total No of Fishes are: " + str(count), (50, 50), cv2.FONT_HERSHEY_SIMPLEX, 0.7, (0, 0, 255), 1, cv2.LINE_AA)

cv2_imshow(image_predict)
cv2.waitKey(0)
cv2.destroyAllWindows()

print("Total Shrimps are:", count)

In [15]:
y2

1008.385498046875

In [ ]:
import pandas as pd
pd.DataFrame(results.boxes.data.tolist())

,0,1,2,3,4,5
0,977.349243,783.147888,1165.875488,895.596619,0.623353,0.0
1,1258.901367,680.739441,1415.098877,787.138977,0.558278,0.0
2,213.821030,764.154968,362.106018,870.513428,0.404191,0.0
3,363.970184,882.003052,612.777161,1008.385498,0.390487,0.0
